In [3]:
!pip install -r requirements.txt --no-warn-conflicts --progress-bar emoji
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import girder_client
import json
import os
import pandas as pd
import psycopg2
import postgres_to_girder
import urllib
from datetime import date

config, context, api_url = postgres_to_girder.config()
girder_connection=postgres_to_girder.connect_to_girder(
    api_url=api_url,
    authentication=(
        config["girder"]["user"],
        config["girder"]["password"]
    )
)
conn = postgres_to_girder.connect_to_postgres(
    config["postgres"]
)

Connected to the Girder database 🏗🍃 and authenticated.
Connected to the Postgres database 🐘


In [7]:
activities_id = postgres_to_girder.get_girder_id_by_name(
    entity="collection",
    name="activities",
    girder_connection=girder_connection
)
activities_id = girder_connection.createCollection(
    name="activities",
    public=True
) if not activities_id else activities_id

In [11]:
postgres_tables = {
    table: pd.io.sql.read_sql_query(
        "SELECT * FROM {0};".format(
            table
        ),
        conn
    ) for table in {
        "acts",
        "users",
        "user_acts",
        "organizations",
        "answers"
    }
}

Pull respondents out of titles in DataFrame from Postgres:

In [ ]:
acts["Respondent"] = acts["title"].apply(
    lambda x: x.split(
        " - "
    )[
        1
    ].split(
        " "
    )[
        0
    ] if " - " in x else x.split(
        " – "
    )[
        1
    ].split(
        " "
    )[
        0
    ] if " – " in x else x.split(
        "-"
    )[
        1
    ].split(
        " "
    )[
        0
    ] if "Scale-" in x else x.split(
        " ― "
    )[
        1
    ].split(
        "-"
    )[
        0
    ] if "―" in x else x.split(
        "-"
    )[
        1
    ].split(
        ")"
    )[
        0
    ] if "Index-" in x else "Self" if (
        (
            "_SR" in x
        ) or (
            "-SR" in x
        )
    ) else "Parent" if (
        "_P" in x
    ) else ""
)
acts["title"] = acts["title"].apply(
    lambda x: x.split(
        " - "
    )[
        0
    ] if " - " in x else x.split(
        " – "
    )[
        0
    ] if " – " in x else x.split(
        "-"
    )[
        0
    ] if "Scale-" in x else x.split(
        " ― "
    )[
        0
    ] if "―" in x else x.split(
        "-"
    )[
        0
    ] if "Index-" in x else x.replace(
        " Self Report",
        ""
    ).replace(
        " Parent Report",
        ""
    )
).apply(
    lambda x: "{0})".format(
        x
    ) if (
        "(" in x
    ) and (
        ")"
    ) not in x else x
)

In [ ]:
acts

In [ ]:
for i, activity in enumerate(
    list(
        acts["title"]
    )
):
    abbreviation = None
    if "(" in activity:
        anames = [
            a.strip(
                ")"
            ).strip() for a in activity.split(
                "("
            )
        ]
        if (
            len(anames)==2
        ):
            if (
                len(anames[0])>len(anames[1])
            ):
                abbreviation = anames[1]
                activity_name = anames[0]
            else:
                abbreviation = anames[0]
                activity_name = anames[1]
        else:
            print(anames)
    activity_name = activity if not abbreviation else activity_name
            
    respondent = acts[
        acts["title"]==activity
    ]["Respondent"].values[0]
    item_version = "{0} ({1})".format(
        " ― ".join([
            "Healthy Brain Network",
            "{0} ({1}) {2}".format(
                activity_name,
                abbreviation,
                "― {0} Report".format(
                    respondent
                ) if len(respondent) else ""
            ) if abbreviation else "{0} {1}".format(
                activity,
                "― {0} Report".format(
                    respondent
                ) if len(respondent) else ""
            )
        ]).strip(" "),
        date.strftime(
            acts.loc[
                i,
                "updated_at"
            ],
            "%F"
        )
    )
        
    # Create or locate top-level folder and return _id
    activity_folder_id = gc.createFolder(
        name=activity if not abbreviation else activity_name,
        parentId=activities_id,
        parentType="collection",
        public=True,
        reuseExisting=True
    )["_id"]
    
    # Create or locate Item
    user = {
        "@id": "".join([
            "user/",
            postgres_to_girder.get_user_id_by_email(
                gc,
                users[
                    users["id"]==acts.loc[
                        i,
                        "user_id"
                    ]
                ]["email"].values[0]
            )
        ])
    }
    act_data = json.loads(
        acts.loc[
            i,
            "act_data"
        ]
    )
    activity_item_id = gc.createItem(
        name=item_version,
        parentFolderId=activity_folder_id,
        reuseExisting=True,
        metadata={
            **context,
            "schema:name": {
                "@value": activity_name,
                "@language": "en-US"
            },
            "abbreviation": abbreviation if abbreviation else None,
            "@type": acts.loc[
                i,
                "type"
            ],
            "status": acts.loc[
                i,
                "status"
            ],
            "pav:lastUpdatedOn": acts.loc[
                i,
                "updated_at"
            ].isoformat(),
            **{ prop: act_data[prop] for prop in 
                act_data if prop not in [
                    "questions",
                    "instruction",
                    "image_url"
                ]
            },
            "instruction": {
                "@value": act_data["instruction"],
                "@language": "en-US"
            } if (
                (
                    "instruction" in act_data
                ) and len(
                    act_data["instruction"]
                )
            ) else None,
            "oslc:modifiedBy": user,
            "pav:createdBy": user
        }
    )["_id"]
        
    # Upload applicable file(s)
    img = urllib.request.urlopen(
        act_data["image_url"]
    ) if "image_url" in act_data else None
    if img:
        item_files = \
        postgres_to_girder.get_files_in_item(
            gc,
            activity_item_id
        )
        img_name = ".".join([
            ''.join([
                c for c in activity_name if \
                c.isalnum()
            ]),
            act_data[
                "image_url"
            ].split("?")[0].split(".")[-1]
        ])
        img_id = item_files[
            0
        ][
            "_id"
        ] if img_name in [
            file[
                "name"
            ] for file in \
            item_files
        ] else gc.uploadFile(
            parentId=activity_item_id,
            stream=img,
            name=img_name,
            size=int(img.info()["Content-Length"])
        )["_id"]
        gc.addMetadataToItem(
            itemId=activity_item_id,
            metadata={
                "image_url": "".join([
                    api_url,
                    "/file/",
                    new_img_id,
                    "/download?",
                    "contentDisposition="
                    "inline"
                ])
            }
        )

In [16]:
def postgres_users_to_girder_users(
    users,
    girder_connection,
    unknown_person={
        "first_name": "Notname",
        "last_name": "Anonymous"
    }
):
    """
    Function to transfer users from Postgres table to
    Girder collection.
    
    Parameters
    ----------
    users: DataFrame
        users table from Postgres DB
        
    girder_connection: GirderClient
        active GirderClient in which to add the users
        
    unknown_person: dictionary
        unknown_person["first_name"]: string
        unknown_person["last_name"]: string
    
    Returns
    -------
    users_by_email: dictionary
        key: string
            email address
        value: string
            Girder User_id
    
    Example
    -------
    >>> import girder_client as gc
    >>> import pandas as pd
    >>> postgres_users_to_girder_users(
    >>>     pd.DataFrame(
    ...         {
    ...             'first_name': ['Packages,'],
    ...             'last_name': ['TravisCI'],
    ...             'email': ['travis-packages'],
    ...             'password': [
    ...                 '$2b$12$jchNQFK2jj7UZ/papXfWsu1'
    ...                 '6enEeCUjk2gUxWJ/n6iYPYmejcmNnq'
    ...             ],
    ...             'role': ['user']
    ...         }
    ...     ),
    ...     girder_connection=gc.GirderClient(
    ...         apiUrl="https://data.kitware.com/api/v1/"
    ...     )
    ... )
    {'travis-packages': '55535d828d777f082b592f54'}
    """
    users_by_email = {}
    for i in range(users.shape[0]):
        user_id = postgres_to_girder.get_user_id_by_email(
            girder_connection,
            users.loc[i,"email"]
        )
        if user_id:
            users_by_email[
                users.loc[i,"email"]
            ] = user_id
        else: # pragma: no test
            users_by_email[
                users.loc[i,"email"]
            ] = girder_connection.post(
                "".join([
                    "user?login=",
                    users.loc[i,"email"].replace(
                        "@",
                        "at"
                    ),
                    "&firstName=",
                    unknown_person[
                        "first_name"
                    ] if not users.loc[
                        i,
                        "first_name"
                    ] else users.loc[
                        i,
                        "first_name"
                    ] if not " " in users.loc[
                        i,
                        "first_name"
                    ] else users.loc[
                        i,
                        "first_name"
                    ].split(" ")[0],
                    "&lastName=",
                    users.loc[
                        i,
                        "last_name"
                    ] if users.loc[
                        i,
                        "last_name"
                    ] else unknown_person[
                        "last_name"
                    ] if not users.loc[
                        i,
                        "first_name"
                    ] else users.loc[
                        i,
                        "first_name"
                    ].split(" ")[1] if " " in users.loc[
                        i,
                        "first_name"
                    ] else users.loc[
                        i,
                        "first_name"
                    ],
                    "&password=",
                    users.loc[i,"password"],
                    "&admin=",
                    "true" if "admin" in str(
                        users.loc[
                            i,
                            "role"
                        ]
                    ) else "false",
                    "&email=",
                    users.loc[
                        i,
                        "email"
                    ]
                ])
            ) # pragma: no test
    return(users_by_email)

In [17]:
postgres_users_to_girder_users(
    postgres_tables["users"],
    girder_connection,
    config["missing_persons"]
)

{'erik@test.com': '5b2167b9ad97092bdb8e9cd3',
 'Erik2@test.com': '5b2167b9ad97092bdb8e9cd6',
 'admin@mindlogger.org': '5b2167baad97092bdb8e9cd9',
 'arno@binarybottle.com': '5b2167baad97092bdb8e9cdc',
 'arno@childmind.org': '5b2167baad97092bdb8e9cdf',
 'arno@mindboggle.info': '5b2167bbad97092bdb8e9ce2',
 'admin@ab2cd.com': '5b19830ead97092bdb8e99ed',
 'jon.clucas@chilldmind.org': '5b218204ad97092bdb8e9ceb',
 'jon@vasegurt.com': '5b198293ad97092bdb8e99ea',
 'anirudh.krishnakumar@childmind.org': '5b2167bbad97092bdb8e9ce8'}

In [57]:
"$2b$12$jchNQFK2jj7UZ/papXfWsu16enEeCUjk2gUxWJ/n6iYPYmejcmNnq"[30:]

'6enEeCUjk2gUxWJ/n6iYPYmejcmNnq'

In [54]:
postgres_users_to_girder_users(
    pd.DataFrame(
        {
            'first_name': ['Packages,'],
            'last_name': ['TravisCI'],
            'email': ['travis-packages'],
            'password': ['$2b$12$jchNQFK2jj7UZ/papXfWsu16enEeCUjk2gUxWJ/n6iYPYmejcmNnq'],
            'role': ['user']
        }
    ),
    girder_connection=girder_client.GirderClient(
        apiUrl="https://data.kitware.com/api/v1/"
    )
)

{'travis-packages': '55535d828d777f082b592f54'}